# 💻 WAE Chatbot Workshop

## Installing Required Libraries
We will use:

Gradio for building simple user interfaces.

Mistral AI for accessing the language model.


In [ ]:
!pip install mistralai
!pip install gradio

from IPython.display import clear_output
clear_output() # Clear previous output for a cleaner notebook

Our API is currently available through [La Plateforme](https://console.mistral.ai/). You need to activate payments on your account to enable your API keys. After activation, you can start using the `chat` endpoint to interact with models.

##  Setting up the Mistral Model

You will need to add your API Key from:  https://console.mistral.ai/api-keys/


You can explore all available models here: https://docs.mistral.ai/getting-started/models/models_overview/



In [ ]:
from mistralai import Mistral

api_key = "YOUR_API_KEY_HERE" # 👉 Replace with your own API Key
model = "mistral-large-2402"  # You can change the model if you want

client = Mistral(api_key=api_key)

# Building a Simple Chatbot using Mistral Model

In [ ]:
def chat_with_mistral(message, history):
    # Build the conversation context by combining the existing history with the new user message.
    messages = history + [{"role": "user", "content": message}]

    chat_response = client.chat.complete(
        model=model,
        messages=messages # Content from the user input
    )
    response = chat_response.choices[0].message.content

    formatted_response = {"role": "assistant", "content": response}

    # Return only the new assistant message.
    return formatted_response

# Creating a Web Interface using Gradio

In [ ]:
import gradio as gr

iface = gr.ChatInterface(
    fn=chat_with_mistral, # Function to handle chat messages
    title="Mistral AI Chatbot",
    description="Chat with the Mistral AI large language model.",
    theme="glass", # Visual theme
    type="messages" # Set the input/output format to handle full chat messages
)

iface.launch(
    debug=True, # Show detailed error messages if something goes wrong
    share=True, # Create a public link to share your app with others
)

## 🛠 Exercise: Customize Your Chatbot’s Personality!

In [ ]:
# PROVIDE SOME STARTER CODE TO GUIDE STUDENTS

#### **hint:**

 {"role": "system", "content": "Your special instruction goes here."}

# Building a Chatbot using Hugging Face Transformers

In [ ]:
!pip install transformers
!pip install torch

from IPython.display import clear_output
clear_output()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "facebook/opt-125m"  # Choose a chatbot model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name) # tokenizer: converts text into tokens that the model can understand
model = AutoModelForCausalLM.from_pretrained(model_name) # model: generates text responses based on the input tokens

In [ ]:
def chat_with_huggingface(user_input, history):
    # Tokenize the new user input
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Build input by concatenating history (if any) and new user input
    if history:
        # Convert history list to tensor and add batch dimension
        history_tensor = torch.tensor(history).unsqueeze(0)
        bot_input_ids = torch.cat([history_tensor, new_user_input_ids], dim=-1)
    else:
        bot_input_ids = new_user_input_ids

    # Create an attention mask (1 for non-eos_token, 0 for eos_token)
    attention_mask = (bot_input_ids != tokenizer.eos_token_id).long()

    # Generate a response
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=100, # Limit the response length
        pad_token_id=tokenizer.eos_token_id, # Padding token (end of sequence)
        attention_mask=attention_mask, # Mask to focus on relevant tokens
        repetition_penalty=1.2,      # discourages repeating the same phrases
        no_repeat_ngram_size=3,      # prevents repeating n-grams of a given size
        do_sample=True,              # enables sampling (instead of greedy decoding)
        top_p=0.9,                   # controls diversity
        temperature=0.7              # Controls randomness in generation (lower = less random)
    )


    # Extract only the new tokens generated for the response
    new_tokens = chat_history_ids[:, bot_input_ids.shape[-1]:][0]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)

    # Update history with both user input tokens and assistant's response tokens
    history.extend(new_user_input_ids.tolist()[0])
    history.extend(new_tokens.tolist())

    return response, history


In [ ]:
# Initialize conversation history as an empty list (no previous messages)
chat_history = []

# Example usage of the chatbot function with a user input
user_input = "Hello, how are you?"
response, chat_history = chat_with_huggingface(user_input, chat_history)

# Print the chatbot's response
print(f"Chatbot: {response}")

## 🛠 Exercise: Try a Different Hugging Face Model!

In [ ]:
# PROVIDE SOME STARTER CODE TO GUIDE STUDENTS

# 🎉 Congratulations

Congratulations on completing the workshop!

Today, you learned how to build chatbots using both **Mistral AI** and **Hugging Face Transformers**.

Keep experimenting, explore new models, and never stop learning. 🚀  
The world of AI is full of opportunities — this is just the beginning!

---


This file was created by **Eyad Alfaifi** and **Abdulmalik Alquifly**.